### ETL movies ###

Importamos las librerias necesarias y cargamos el archivo, hacemos una copia para evitar inconsistencias de datos.

In [1]:
import pandas as pd
import unicodedata
import ast

# Carga y copia del dataset
df = pd.read_csv('movies_dataset.csv').copy()

C:\Users\Alberto\AppData\Local\Temp\ipykernel_5292\193078804.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_dataset.csv').copy()


Primer acercamiento a datos

In [2]:
# Verificamos la estructura del DataFrame
print(df.head())

   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497         

Inspeccionamos las columnas para ver que tipo de datos contienen

In [3]:
# Visualizamos la información general del DataFrame (incluye tipos de datos y valores no nulos)
print("Información general del DataFrame:")
print(df.info())

# Verificamos la cantidad de valores nulos en cada columna
print("\nCantidad de valores nulos por columna:")
print(df.isnull().sum())

# Muestra los tipos de datos de todas las columnas
print("\nTipos de datos de cada columna:")
print(df.dtypes)

# Verificamos la cantidad de tipos de datos únicos dentro de cada columna
tipos_por_columna = df.applymap(type).nunique()
print("\nCantidad de tipos de datos únicos por columna:")
print(tipos_por_columna)

Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date       

C:\Users\Alberto\AppData\Local\Temp\ipykernel_5292\752270181.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tipos_por_columna = df.applymap(type).nunique()


Hacemos configuraciones basicas para el tratamiento inicial del dataset

In [4]:
df.drop_duplicates(subset='id', inplace=True) #Borramos duplicados 
df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else None)
df['collection_name'] = df['belongs_to_collection'].apply(lambda x: x.get('name') if isinstance(x, dict) else 'sin colección')
df.drop(columns=['belongs_to_collection'], inplace=True)

Desanidamos las columnas correspondientes

In [5]:
# Función para extraer los datos necesarios
def extract_names(column):
    return column.apply(
        lambda x: [d['name'] for d in ast.literal_eval(x)] 
        if isinstance(x, str) and x.strip().startswith('[') else [])

# Aplicamos la función a las columnas anidadas
df['genre_names'] = extract_names(df['genres'])
df['companies_names'] = extract_names(df['production_companies'])
df['countries_names'] = extract_names(df['production_countries'])
df['spoken_language_names'] = extract_names(df['spoken_languages'])


Procedemos a normalizar los datos en las columnas que contengan texto

In [6]:
# Función para normalizar el texto
def normalizar_texto(texto):
    if isinstance(texto, list):
        return [normalizar_texto(elem) for elem in texto]
    elif isinstance(texto, str):
        return unicodedata.normalize('NFC', texto).lower()
    return texto

# Aplicación de normalización a todas las columnas de texto
df = df.applymap(normalizar_texto)


C:\Users\Alberto\AppData\Local\Temp\ipykernel_5292\1415985429.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalizar_texto)


Tratamiento de datos faltantes

In [7]:
# Rellenamos valores faltantes en columnas numéricas y eliminamos filas sin fecha
df.fillna({'revenue': 0, 'budget': 0}, inplace=True)
df.dropna(subset=['release_date'], inplace=True)


Convertimos la columna 'release_date' a formato datetime y extraemos el año de lanzamiento

In [8]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year
df.dropna(subset=['release_date'], inplace=True)  #Eliminamos valores faltantes


Al tener la necesidad de ir recortando el dataset, tomo la decisión de filtrar por idioma y año de lanzamiento dejando peliculas desde el 2004 en adelante unicamente, ademas de tomar 2300 filas aleatoriamente.

In [9]:
idiomas_permitidos = {'english', 'español', 'portugues'}
df = df[(df['release_year'] >= 2004) &
        (df['spoken_language_names'].apply(lambda x: bool(set(x) & idiomas_permitidos)))]


df = df.sample(n=2300, random_state=42).reset_index(drop=True)  #Reducimos el tamaño del dataset y aseguramos que cada ejecución produzca los mismos resultados gracias al parámetro random_state=42.

Creamos la columna 'return' y ajustamos los tipos de datos a numéricos

In [10]:
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['return'] = df.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)

Para finalizar, eliminamos columnas innecesarias y exportamos el archivo

In [11]:
columns_to_drop = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage',
                   'original_language', 'runtime', 'production_companies', 'production_countries',
                   'tagline', 'genres', 'spoken_languages', 'status']
df.drop(columns=columns_to_drop, inplace=True)

# Exportamos el archivo limpio
df.to_csv("movies_ok.csv", index=False)